<a href="https://colab.research.google.com/github/maya-g-y/Final-Project-ML-Module--Spotify/blob/main/Spotify4_Feature_Engineering_%26_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Feature Engineering & Selection


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Load prepared dataset from pickle (created in Stage 2)
df = pd.read_pickle("/content/drive/MyDrive/pickle_files/final_df_cleansed.pkl")

print("Shape:", df.shape)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape: (30942, 22)


,track_name,track_artist,track_popularity,track_album_name,playlist_name,playlist_genre,danceability,energy,key,loudness,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,release_year,release_month,playlist_subgenre_simplified,playlist_genre_encoded
0,i dont care with justin bieber loud luxury remix,ed sheeran,66.0,i dont care with justin bieber loud luxury remix,pop remix,pop,0.748,0.916,6.0,-2.634,...,0.1020,0.000000,0.0653,0.518,122.036,194754.0,2019.0,6.0,dance pop,2
1,memories dillon francis remix,maroon 5,67.0,memories dillon francis remix,pop remix,pop,0.726,0.815,11.0,-4.969,...,0.0724,0.004210,0.3570,0.693,99.972,162600.0,2019.0,12.0,dance pop,2
2,all the time don diablo remix,zara larsson,70.0,all the time don diablo remix,pop remix,pop,0.675,0.931,1.0,-3.432,...,0.0794,0.000023,0.1100,0.613,124.008,176616.0,2019.0,7.0,dance pop,2
3,call you mine keanu silva remix,the chainsmokers,60.0,call you mine the remixes,pop remix,pop,0.718,0.930,7.0,-3.778,...,0.0287,0.000009,0.2040,0.277,121.956,169093.0,2019.0,7.0,dance pop,2
4,someone you loved future humans remix,lewis capaldi,69.0,someone you loved future humans remix,pop remix,pop,0.650,0.833,1.0,-4.672,...,0.0803,0.000000,0.0833,0.725,123.976,189052.0,2019.0,3.0,dance pop,2


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30942 entries, 0 to 32832
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   track_name                    30942 non-null  string  
 1   track_artist                  30942 non-null  string  
 2   track_popularity              30942 non-null  float64 
 3   track_album_name              30942 non-null  string  
 4   playlist_name                 30942 non-null  string  
 5   playlist_genre                30942 non-null  category
 6   danceability                  30942 non-null  float64 
 7   energy                        30942 non-null  float64 
 8   key                           30942 non-null  float64 
 9   loudness                      30942 non-null  float64 
 10  mode                          30942 non-null  category
 11  speechiness                   30942 non-null  float64 
 12  acousticness                  30942 non-null  float

##Numeric Feature Engineering & Selection

In [3]:
# =========================
# Numeric Feature Engineering & Selection
# =========================

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Define modeling frame
df_model = df.copy()
target_col = "playlist_genre"

# Basic sanity checks
assert target_col in df_model.columns, f"Target column '{target_col}' not found in df.columns"

# Guard against target leakage: explicitly drop encoded target-like cols from features
leakage_cols = [c for c in ["playlist_genre_encoded", "playlist_subgenre_simplified"] if c in df_model.columns]


# High-cardinality / free-text columns — drop
high_card_text = [c for c in ["playlist_name","track_album_name",
                              "track_artist","track_name"] if c in df_model.columns]

# Turn 'mode' (Major/Minor) into a binary flag and drop the original
if "mode" in df_model.columns:
    df_model["mode_major"] = (df_model["mode"].astype(str).str.lower() == "major").astype("int8")
    drop_after_mode = ["mode"]
else:
    drop_after_mode = []


# Apply all drops (never drop the target)
to_drop = [c for c in (leakage_cols + high_card_text + drop_after_mode) if c != target_col]
df_model = df_model.drop(columns=to_drop, errors="ignore")

print("Dropped columns:", to_drop)


# Identify numeric vs categorical columns automatically
numeric_cols = df_model.select_dtypes(include=["number", "bool", "int64", "float64"]).columns.tolist()

# Remove target and leakage columns from numeric list if present
for c in [target_col] + leakage_cols:
    if c in numeric_cols:
        numeric_cols.remove(c)

# Categorical = everything that's not numeric and not target/leakage
categorical_cols = [c for c in df_model.columns if c not in numeric_cols + [target_col] + leakage_cols]

# Features / target split
X = df_model.drop(columns=[target_col])
y = df_model[target_col]

# Safety checks
for c in leakage_cols:
    assert c not in X.columns, f"Leakage column found in X: {c}"
assert target_col not in X.columns, "Target column leaked into X!"


print("Numeric columns (auto-detected):", len(numeric_cols))
print("Categorical columns (auto-detected):", len(categorical_cols))

Dropped columns: ['playlist_genre_encoded', 'playlist_subgenre_simplified', 'playlist_name', 'track_album_name', 'track_artist', 'track_name', 'mode']
Numeric columns (auto-detected): 15
Categorical columns (auto-detected): 0


In [4]:
# Save preprocessed features for reuse (before Feature Engineering & OHE)
X.to_pickle("/content/drive/MyDrive/pickle_files/X_pre_FE.pkl")
print("✅ Features dataset saved as 'X_pre_FE.pkl' in /MyDrive/pickle_files")


✅ Features dataset saved as 'X_pre_FE.pkl' in /MyDrive/pickle_files


In [5]:

# ----- 1) Train/Test split (avoid leakage) -----
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# ----- 2) Feature Engineering function (numeric + time; no text, no 'mode') -----
def add_numeric_features(df_in: pd.DataFrame) -> pd.DataFrame:
    """Create engineered features from existing columns (safe checks included)."""
    out = df_in.copy()
    cols = out.columns

    # ===== Numeric transforms =====
    if "duration_ms" in cols:
        out["duration_min"] = out["duration_ms"] / 60000.0
        out["duration_log"] = np.log1p(out["duration_ms"].clip(lower=0))

    if "tempo" in cols:
        out["tempo_z"] = (out["tempo"] - out["tempo"].mean()) / (out["tempo"].std() + 1e-9)

    if "speechiness" in cols:
        out["speechiness_log"] = np.log1p(out["speechiness"].clip(lower=0))

    if "acousticness" in cols:
        eps = 1e-6
        a = out["acousticness"].clip(eps, 1 - eps)
        out["acousticness_logit"] = np.log(a / (1 - a))

    # ===== Interactions & ratios =====
    # Aligned with EDA findings
    # Energy ↔ Loudness (strong positive)
    if set(["energy", "loudness"]).issubset(cols):
        out["energy_x_loudness"] = out["energy"] * out["loudness"]  # energetic tracks tend to be louder

    # Acousticness ↔ Energy (negative trend; interaction can help linear models capture joint effect)
    if set(["acousticness", "energy"]).issubset(cols):
        out["acousticness_x_energy"] = out["acousticness"] * out["energy"]

    # Danceability ↔ Valence (moderate positive; “rhythmic mood”)
    if set(["danceability", "valence"]).issubset(cols):
        out["danceability_x_valence"] = out["danceability"] * out["valence"]

    # Instrumentalness ↔ Valence and ↔ Speechiness (negative links observed)
    if set(["instrumentalness", "valence"]).issubset(cols):
        out["instrumentalness_x_valence"] = out["instrumentalness"] * out["valence"]
    if set(["instrumentalness", "speechiness"]).issubset(cols):
        out["instrumentalness_x_speechiness"] = out["instrumentalness"] * out["speechiness"]

    # Ratio-based and composite features capturing vocal focus, energy balance, and track complexity
    if "instrumentalness" in cols and "speechiness" in cols:
        out["vocal_focus"] = out["speechiness"] / (out["instrumentalness"] + 1e-6)
    if "energy" in cols and "acousticness" in cols:
        out["energy_ratio"] = out["energy"] / (out["acousticness"] + 1e-6)

    if set(["energy", "loudness", "tempo"]).issubset(cols):
        out["complexity"] = out[["energy", "loudness", "tempo"]].std(axis=1)

    # Overall mood-like index
    if set(["energy", "valence"]).issubset(cols):
        out["mood_index"] = (out["energy"] + out["valence"]) / 2.0

    # ===== Time-based features =====
    if "release_year" in cols:
        current_year = pd.Timestamp.now().year
        out["song_age"] = current_year - out["release_year"]
        out["release_decade"] = (out["release_year"] // 10) * 10
    if "release_month" in cols:
        out["is_summer_release"] = out["release_month"].isin([6, 7, 8]).astype(int)
        out["is_winter_release"] = out["release_month"].isin([12, 1, 2]).astype(int)

    # ===== No text-derived features; no 'mode' handling here =====
    # (track_* and playlist_* were dropped earlier; mode_major already created)

    # Keep index alignment
    return out

print("Before FE:", X_train.shape[1], "features")
X_train_fe = add_numeric_features(X_train)
print("After FE:", X_train_fe.shape[1], "features")
print("New features added:", X_train_fe.shape[1] - X_train.shape[1])

new_features = sorted(set(X_train_fe.columns) - set(X_train.columns))
print("Newly added features:")
print(new_features)

X_train_fe.info()




Before FE: 15 features
After FE: 33 features
New features added: 18
Newly added features:
['acousticness_logit', 'acousticness_x_energy', 'complexity', 'danceability_x_valence', 'duration_log', 'duration_min', 'energy_ratio', 'energy_x_loudness', 'instrumentalness_x_speechiness', 'instrumentalness_x_valence', 'is_summer_release', 'is_winter_release', 'mood_index', 'release_decade', 'song_age', 'speechiness_log', 'tempo_z', 'vocal_focus']
<class 'pandas.core.frame.DataFrame'>
Index: 24753 entries, 2311 to 2289
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   track_popularity                24753 non-null  float64
 1   danceability                    24753 non-null  float64
 2   energy                          24753 non-null  float64
 3   key                             24753 non-null  float64
 4   loudness                        24753 non-null  float64
 5   speechiness         

In [6]:

# Apply FE on train/test (same logic, independently computed per split)
X_train_fe = add_numeric_features(X_train)
X_test_fe  = add_numeric_features(X_test)

# Standardize all numeric features so they share a common scale (mean=0, std=1),
# preventing features with larger numeric ranges from dominating model training.
num_cols_after_fe = X_train_fe.select_dtypes(include=["number", "bool"]).columns.tolist()

scaler = StandardScaler()
X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_fe[num_cols_after_fe]),
    index=X_train_fe.index,
    columns=num_cols_after_fe
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test_fe[num_cols_after_fe]),
    index=X_test_fe.index,
    columns=num_cols_after_fe
)


#Feature Selection

### 🎯 Combined Feature Selection - ANOVA (Univariate) + L1 Logistic (Multivariate)

This step combines two complementary feature selection strategies:

1. **ANOVA F-test (Univariate):**  
   Evaluates each numeric feature individually to identify those that show statistically significant differences between playlist genres.  
   Features with `P-Value < 0.05` are retained as significant predictors.

2. **L1-Regularized Logistic Regression (Multivariate):**  
   Applies a penalized logistic model to the ANOVA-selected features to identify those that contribute most strongly to genre prediction when considered together.

The final selected feature set includes only variables that are both **statistically significant** and **model-relevant**, improving model efficiency and interpretability.


In [7]:
# FEATURE SELECTION — ANOVA F-test (Univariate)

from sklearn.feature_selection import f_classif

# Use only numeric features after FE (same used for L1 selection)
X_train_num = X_train_num_scaled.copy()

# Run ANOVA F-test
f_values, p_values = f_classif(X_train_num, y_train)

anova_df = pd.DataFrame({
    "Feature": X_train_num.columns,
    "F-Value": f_values,
    "P-Value": p_values
}).sort_values("F-Value", ascending=False)

# Display top features
print("Top features by ANOVA F-test:")
display(anova_df.head(15))

Top features by ANOVA F-test:


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [14] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


,Feature,F-Value,P-Value
12,release_year,1913.184326,0.0
29,song_age,1913.184326,0.0
30,release_decade,1832.531149,0.0
18,speechiness_log,1221.586927,0.0
5,speechiness,1182.061248,0.0
1,danceability,1058.679659,0.0
25,vocal_focus,906.983152,0.0
2,energy,835.272225,0.0
19,acousticness_logit,772.856751,0.0
7,instrumentalness,487.071103,0.0


In [8]:
anova_df.style.background_gradient(cmap="coolwarm", subset=["F-Value"])\
               .format({"P-Value": "{:.2e}", "F-Value": "{:.0f}"})


,Feature,F-Value,P-Value
12,release_year,1913,0.00e+00
29,song_age,1913,0.00e+00
30,release_decade,1833,0.00e+00
18,speechiness_log,1222,0.00e+00
5,speechiness,1182,0.00e+00
1,danceability,1059,0.00e+00
25,vocal_focus,907,0.00e+00
2,energy,835,0.00e+00
19,acousticness_logit,773,0.00e+00
7,instrumentalness,487,0.00e+00


In [9]:
# Keep only statistically significant features (p < 0.05)
significant_features = anova_df.loc[anova_df["P-Value"] < 0.05, "Feature"].tolist()
print(f"Significant features (p < 0.05): {len(significant_features)} retained out of {len(anova_df)}")

# Filter X to only those features
X_train_anova = X_train_num_scaled[significant_features].copy()
X_test_anova  = X_test_num_scaled[significant_features].copy()

# L1 Logistic Regression (Multivariate Selection)
le_y = LabelEncoder()
y_train_enc = le_y.fit_transform(y_train)

fs_model = LogisticRegression(
    penalty="l1",
    solver="saga",
    multi_class="multinomial",
    C=1.0,
    max_iter=5000,
    n_jobs=-1
)

selector = SelectFromModel(
    estimator=fs_model,
    threshold="median",   # keep features with |coef| above the median
    prefit=False
)

selector.fit(X_train_anova, y_train_enc)

mask = selector.get_support()
selected_features_final = list(np.array(significant_features)[mask])

# Print summary
print("Total numeric after FE:", X_train_num_scaled.shape[1])
print("After ANOVA (p<0.05):", len(significant_features))
print("After ANOVA + L1:", len(selected_features_final))
print("Final selected features:")
print(selected_features_final)

#  Keep only selected features for downstream modeling
X_train_final = X_train_num_scaled[selected_features_final].copy()
X_test_final  = X_test_num_scaled[selected_features_final].copy()

print("\nShapes | train:", X_train_final.shape, "| test:", X_test_final.shape)

Significant features (p < 0.05): 32 retained out of 33


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Total numeric after FE: 33
After ANOVA (p<0.05): 32
After ANOVA + L1: 16
Final selected features:
[np.str_('release_year'), np.str_('song_age'), np.str_('release_decade'), np.str_('speechiness_log'), np.str_('speechiness'), np.str_('danceability'), np.str_('vocal_focus'), np.str_('energy'), np.str_('acousticness_logit'), np.str_('loudness'), np.str_('danceability_x_valence'), np.str_('valence'), np.str_('duration_log'), np.str_('duration_ms'), np.str_('duration_min'), np.str_('energy_x_loudness')]

Shapes | train: (24753, 16) | test: (6189, 16)


### Feature Selection Summary

Out of 33 numeric features:  
- **32** passed the ANOVA test (p < 0.05)  
- **16** were retained after ANOVA + L1 Logistic  

Final features include key temporal, acoustic, and emotional traits  
(e.g., `energy`, `loudness`, `valence`, `danceability`, `release_year`).  
These represent the most predictive variables for playlist genre.





In [ ]:
# =========================
# MODEL-CONSENSUS FEATURE SELECTION (for classification)
# Uses: Logistic L1, LinearSVC L1, GradientBoost, RandomForest
# =========================

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import f_classif

# ---------- 0) Base matrix: numeric after FE+scaling ----------
Xn_train = X_train_num_scaled.copy()
Xn_test  = X_test_num_scaled.copy()
feat_all = Xn_train.columns.tolist()

# ---------- 1) ANOVA filter (use existing anova_df if available; otherwise compute) ----------
if 'anova_df' not in globals():
    f_vals, p_vals = f_classif(Xn_train, y_train)
    anova_df = pd.DataFrame({"Feature": feat_all, "F-Value": f_vals, "P-Value": p_vals}).sort_values("F-Value", ascending=False)

# Option A: p-value threshold
p_thresh = 0.05
candidates = anova_df.loc[anova_df["P-Value"] < p_thresh, "Feature"].tolist()

# If too many/too few, optionally cap by top-K F-values
if len(candidates) == 0:
    # fallback: take top 40 by F
    candidates = anova_df["Feature"].head(40).tolist()
elif len(candidates) > 80:
    # optional cap to keep things fast
    candidates = anova_df.sort_values("F-Value", ascending=False).head(80)["Feature"].tolist()

Xc_train = Xn_train[candidates].astype("float32").copy()
Xc_test  = Xn_test[candidates].astype("float32").copy()

print(f"Candidates after ANOVA: {len(candidates)} / {len(feat_all)} features")

# ---------- 2) Encode y for linear models ----------
le_y = LabelEncoder()
y_train_enc = le_y.fit_transform(y_train)

# ---------- 3) Define models ----------
log_l1 = LogisticRegression(
    penalty="l1", solver="saga", multi_class="ovr",
    class_weight="balanced",
    C=1.0, max_iter=2000, tol=1e-3, n_jobs=-1
)

svm_l1 = LinearSVC(
    C=0.1, penalty="l1", dual=False
)  # l1 requires dual=False; multi-class handled as OvR

gb = GradientBoostingClassifier(random_state=42)
rf = RandomForestClassifier(
    n_estimators=300, random_state=42, n_jobs=-1,
    class_weight="balanced_subsample"
)

# ---------- 4) Fit & build per-model selection masks ----------
votes = pd.DataFrame({"Feature": candidates}).set_index("Feature")

# 4a) Logistic L1 — non-zero max |coef| across classes
log_l1.fit(Xc_train, y_train_enc)
coef_log = np.abs(log_l1.coef_)             # shape: (n_classes, n_features)
imp_log  = coef_log.max(axis=0)             # aggregate across classes
votes["Logistic_L1"] = (imp_log > 0).astype(int)

# 4b) LinearSVC L1 — non-zero max |coef| across classes
svm_l1.fit(Xc_train, y_train_enc)
coef_svm = np.abs(svm_l1.coef_)             # OvR → shape: (n_classes, n_features)
imp_svm  = coef_svm.max(axis=0)
votes["LinearSVC_L1"] = (imp_svm > 0).astype(int)

# 4c) Gradient Boosting — use importance; be a bit selective (above median)
gb.fit(Xc_train, y_train)
imp_gb = gb.feature_importances_
thr_gb = np.median(imp_gb)                  # avoids selecting “everything”
votes["GradientBoost"] = (imp_gb > thr_gb).astype(int)

# 4d) Random Forest — use importance; be a bit selective (above median)
rf.fit(Xc_train, y_train)
imp_rf = rf.feature_importances_
thr_rf = np.median(imp_rf)
votes["RandomForest"] = (imp_rf > thr_rf).astype(int)

# ---------- 5) Consensus: keep features with >= MIN_VOTES ----------
votes["Sum"] = votes[["Logistic_L1", "LinearSVC_L1", "GradientBoost", "RandomForest"]].sum(axis=1)
MIN_VOTES = 3  # require agreement by at least 3 models (tune as you wish)

selected_by_committee = votes.index[votes["Sum"] >= MIN_VOTES].tolist()
selected_by_committee = sorted(selected_by_committee, key=lambda f: votes.loc[f, "Sum"], reverse=True)

print(f"Selected by committee (>= {MIN_VOTES} votes): {len(selected_by_committee)} / {len(candidates)}")
display(votes.sort_values("Sum", ascending=False).head(20))

# ---------- 6) Build final numeric matrices from committee selection ----------
X_train_committee = Xn_train[selected_by_committee].copy()
X_test_committee  = Xn_test[selected_by_committee].copy()
print("Shapes — committee numerics | train:", X_train_committee.shape, "| test:", X_test_committee.shape)


Candidates after ANOVA: 31 / 32 features


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Selected by committee (>= 3 votes): 16 / 31


,Logistic_L1,LinearSVC_L1,GradientBoost,RandomForest,Sum
Feature,,,,,
release_year,1,1,1,1,4
song_age,1,1,1,1,4
speechiness_log,1,1,1,1,4
danceability,1,1,1,1,4
speechiness,1,1,1,1,4
vocal_focus,1,1,1,1,4
energy,1,1,1,1,4
valence,1,1,1,1,4
energy_x_danceability,1,1,1,1,4


Shapes — committee numerics | train: (24753, 16) | test: (6189, 16)


### 🔗 Combining Feature Selection Results - Intersection vs. Union

To compare and consolidate results from different feature selection strategies,  
two sets of features are combined:

- **Intersection (`both`)** – keeps only features that were selected **by both methods**  
  (i.e., by the ANOVA + L1 Logistic approach *and* by the model committee).  
  This produces a **conservative** and interpretable feature subset, focusing on the most consistently important predictors.

- **Union (`either`)** – includes all features that were selected **by at least one method**.  
  This creates a **broader** feature space that may capture additional nonlinear or model-specific relationships.

Using both sets allows for testing trade-offs between model simplicity and predictive performance:
- `both` → fewer, more stable and interpretable features.  
- `either` → more features, potentially higher accuracy but higher risk of overfitting.


In [ ]:
both = sorted(list(set(selected_by_committee).intersection(set(selected_features_final))))
either = sorted(list(set(selected_by_committee).union(set(selected_features_final))))


In [ ]:
# =========================
# Compare selected features — Intersection vs. Union (robust & self-contained)
# =========================
import matplotlib.pyplot as plt
import seaborn as sns

# ---- Resolve inputs ----
# Expect either 'both'/'either' already defined OR compute them from the two source lists:
if 'both' not in globals() or 'either' not in globals():
    # Try to compute from selected_by_committee and selected_features_final
    sel_comm = set(selected_by_committee) if 'selected_by_committee' in globals() else set()
    sel_final = set(selected_features_final) if 'selected_features_final' in globals() else set()
    both = sorted(list(sel_comm & sel_final))
    either = sorted(list(sel_comm | sel_final))

# ---- Build comparison dataframe (keep boolean columns!) ----
all_feats = sorted(list(set(both) | set(either)))
comparison_df = pd.DataFrame({"Feature": all_feats})
comparison_df["In_Intersection"] = comparison_df["Feature"].isin(both)
comparison_df["In_Union"] = comparison_df["Feature"].isin(either)

# Human-readable label (do NOT drop the boolean cols)
comparison_df["Selection_Type"] = np.select(
    [
        comparison_df["In_Intersection"] & comparison_df["In_Union"],   # appears in both
        ~comparison_df["In_Intersection"] & comparison_df["In_Union"]   # only in union
    ],
    ["✅ In Both (Intersection)", "➕ Only in Union"],
    default="❌ Not Selected"
)

print(f"Intersection features: {len(both)}")
print(f"Union features: {len(either)}")

# Show full table (including boolean flags)
display(comparison_df[["Feature", "Selection_Type", "In_Intersection", "In_Union"]])





Intersection features: 7
Union features: 25


,Feature,Selection_Type,In_Intersection,In_Union
0,acousticness_logit,➕ Only in Union,False,True
1,complexity,➕ Only in Union,False,True
2,danceability,✅ In Both (Intersection),True,True
3,duration_log,➕ Only in Union,False,True
4,duration_min,➕ Only in Union,False,True
5,duration_ms,➕ Only in Union,False,True
6,energy,✅ In Both (Intersection),True,True
7,energy_ratio,➕ Only in Union,False,True
8,energy_x_danceability,✅ In Both (Intersection),True,True
9,energy_x_valence,➕ Only in Union,False,True


### Categorical Encoding (Conditionally Skipped) & Final Assembly

All remaining features are numeric after prior cleanup (high-cardinality text was dropped and `mode` → `mode_major`).  
Therefore, **no One-Hot Encoding is required** and the final modeling matrices are built directly from the **selected numeric features**.  
If categorical columns reappear in future runs, the pipeline will automatically encode them and concatenate with the numeric block.


In [ ]:
# =========================
# Categorical Encoding (skipped if none) and Final Dataset Assembly
# =========================

# If there are no remaining categorical columns, skip OHE gracefully
if "categorical_cols" in globals() and len(categorical_cols) > 0:
    from sklearn.preprocessing import OneHotEncoder
    print(f"Categorical columns to encode: {len(categorical_cols)}")
    display(categorical_cols)

    ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore', min_frequency=None)

    X_train_cat_ohe = pd.DataFrame(
        ohe.fit_transform(X_train[categorical_cols]),
        index=X_train.index,
        columns=ohe.get_feature_names_out(categorical_cols)
    )
    X_test_cat_ohe = pd.DataFrame(
        ohe.transform(X_test[categorical_cols]),
        index=X_test.index,
        columns=ohe.get_feature_names_out(categorical_cols)
    )

    print("Encoded categorical features:", X_train_cat_ohe.shape[1])

    # Combine numeric + categorical (final features)
    X_train_final_full = pd.concat([X_train_final, X_train_cat_ohe], axis=1)
    X_test_final_full  = pd.concat([X_test_final,  X_test_cat_ohe ], axis=1)

else:
    print("No categorical columns to encode — skipping OHE.")
    # Use numeric-selected features as the final modeling matrices
    X_train_final_full = X_train_final.copy()
    X_test_final_full  = X_test_final.copy()

print("Final training shape:", X_train_final_full.shape)
print("Final testing shape:",  X_test_final_full.shape)


No categorical columns to encode — skipping OHE.
Final training shape: (24753, 16)
Final testing shape: (6189, 16)


In [ ]:
# =========================
# Inspect final modeling datasets
# =========================

print("🟩 TRAIN SET INFO")
print("="*40)
print(f"Shape: {X_train_final_full.shape}")
print(f"Target shape: {y_train.shape}\n")
X_train_final_full.info()

print("\n\n🟦 TEST SET INFO")
print("="*40)
print(f"Shape: {X_test_final_full.shape}")
print(f"Target shape: {y_test.shape}\n")
X_test_final_full.info()

# Optional: quick sanity check for alignment
assert X_train_final_full.shape[1] == X_test_final_full.shape[1], "❌ Feature mismatch between train/test!"
print("\n✅ Train/Test feature sets are aligned correctly.")


🟩 TRAIN SET INFO
Shape: (24753, 16)
Target shape: (24753,)

<class 'pandas.core.frame.DataFrame'>
Index: 24753 entries, 2311 to 2289
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   release_year           24753 non-null  float64
 1   song_age               24753 non-null  float64
 2   release_decade         24753 non-null  float64
 3   speechiness_log        24753 non-null  float64
 4   speechiness            24753 non-null  float64
 5   danceability           24753 non-null  float64
 6   energy                 24753 non-null  float64
 7   acousticness_logit     24753 non-null  float64
 8   energy_x_danceability  24753 non-null  float64
 9   loudness               24753 non-null  float64
 10  energy_x_valence       24753 non-null  float64
 11  duration_log           24753 non-null  float64
 12  duration_ms            24753 non-null  float64
 13  duration_min           24753 non-null  float64
 1

In [ ]:
# Save final encoded datasets
X_train_final_full.to_pickle("/content/drive/MyDrive/pickle_files/X_train_final_full.pkl")
X_test_final_full.to_pickle("/content/drive/MyDrive/pickle_files/X_test_final_full.pkl")
y_train.to_pickle("/content/drive/MyDrive/pickle_files/y_train.pkl")
y_test.to_pickle("/content/drive/MyDrive/pickle_files/y_test.pkl")

print("✅ One-Hot Encoding complete - final data saved.")

✅ One-Hot Encoding complete - final data saved.
